# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [21]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [22]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,mhamid,29.8200,-5.7200,17.75,23,0,1.62,MA,1710376756
1,1,port-aux-francais,-49.3500,70.2167,5.47,84,100,3.97,TF,1710376968
2,2,mount gambier,-37.8333,140.7667,19.53,47,2,5.12,AU,1710376968
3,3,petropavlovsk-kamchatsky,53.0452,158.6483,-0.67,86,100,22.35,RU,1710376968
4,4,annau,37.8876,58.5160,3.72,81,11,2.06,TM,1710376968


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [23]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    title = "OSM", 
    frame_width = 800,
    frame_height = 500,
    size = "Humidity", 
    scale = 1,
    color = "City",
    alpha = 0.5)

# Display the map
map_plot

:Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [24]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_conditions = city_data_df.loc[(city_data_df["Max Temp"] <= 27) & (city_data_df["Max Temp"] >= 21)&
                                    (city_data_df["Wind Speed"] <= 4.5) & (city_data_df["Cloudiness"] == 0)]

# Drop any rows with null values
ideal_conditions_df = ideal_conditions.dropna(axis=0, how="any")

# Display sample data
ideal_conditions_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
90,90,taoudenni,22.6783,-3.9836,24.48,15,0,1.70,ML,1710376994
128,128,ingraj bazar,25.0000,88.1500,22.66,35,0,0.59,IN,1710377009
175,175,partur,19.6000,76.2167,22.65,27,0,2.89,IN,1710377022
177,177,satkhira,22.7167,89.1000,24.86,83,0,1.79,BD,1710377023
182,182,arraial do cabo,-22.9661,-42.0278,23.98,83,0,1.03,BR,1710376779


### Step 3: Create a new DataFrame called `hotel_df`.

In [25]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_conditions_df[["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]=""

# Display sample data
hotel_df

/var/folders/65/g3fq_xgj383gkz0z__dwst4w0000gn/T/ipykernel_12782/730792372.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"]=""


,City,Country,Lat,Lng,Humidity,Hotel Name
90,taoudenni,ML,22.6783,-3.9836,15,
128,ingraj bazar,IN,25.0000,88.1500,35,
175,partur,IN,19.6000,76.2167,27,
177,satkhira,BD,22.7167,89.1000,83,
182,arraial do cabo,BR,-22.9661,-42.0278,83,
194,salalah,OM,17.0151,54.0924,73,
217,rio grande,BR,-32.0350,-52.0986,91,
263,marco,US,25.9726,-81.7290,76,
316,aswan,EG,24.0934,32.9070,7,
378,marco island,US,25.9363,-81.7157,76,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [26]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories": "accomodataion.hotel",
          "limit": 1,
          "apiKey": geoapify_key
   
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude=row["Lat"]
    longitude=row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude}{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
taoudenni - nearest hotel: No hotel found
ingraj bazar - nearest hotel: No hotel found
partur - nearest hotel: No hotel found
satkhira - nearest hotel: No hotel found
arraial do cabo - nearest hotel: No hotel found
salalah - nearest hotel: No hotel found
rio grande - nearest hotel: No hotel found
marco - nearest hotel: No hotel found
aswan - nearest hotel: No hotel found
marco island - nearest hotel: No hotel found
bagula - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
90,taoudenni,ML,22.6783,-3.9836,15,No hotel found
128,ingraj bazar,IN,25.0000,88.1500,35,No hotel found
175,partur,IN,19.6000,76.2167,27,No hotel found
177,satkhira,BD,22.7167,89.1000,83,No hotel found
182,arraial do cabo,BR,-22.9661,-42.0278,83,No hotel found
194,salalah,OM,17.0151,54.0924,73,No hotel found
217,rio grande,BR,-32.0350,-52.0986,91,No hotel found
263,marco,US,25.9726,-81.7290,76,No hotel found
316,aswan,EG,24.0934,32.9070,7,No hotel found
378,marco island,US,25.9363,-81.7157,76,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [27]:
%%capture --no-display

# Configure the map plot
map_plot_2=hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    title = "OSM",
    frame_width = 800,
    frame_height = 500,
    size = "Humidity",
    scale = 2,
    color = "City",
    alpha = 0.5,
    hover_cols = ["Hotel Name", "Country"])

# Display the map
map_plot_2

:Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)